In [ ]:
!pip install fastText
!pip install datasets

(2 points) Turn the dataset into a dataset compatible with Fastext (see the Tips on using FastText section a bit lower).

    For pretreatment, only apply lower casing and punctuation removal.


In [ ]:
import fasttext
from datasets import load_dataset

dataset_train = load_dataset('imdb', split='train')

In [ ]:
import random
random.seed(42)

In [ ]:
from string import punctuation


def to_lower(review: str):
	return str.lower(review)


def no_punct(review: str):
	return review.translate(str.maketrans('', '', punctuation))


def to_fasttext(review: str, label: int):

	label_str = 'positive'
	if label == 0:
		label_str = 'negative'
	return '__label__' + label_str + " " + review


def pretreatment(dataset: dict):
	labels = dataset['label']
	reviews = dataset['text']
	corrected_review_list = []
	for i in range(len(reviews)):
		lower = to_lower(reviews[i])
		no_punc = no_punct(lower)
		corrected_review = to_fasttext(no_punc, labels[i])
		corrected_review_list.append(corrected_review)
	return corrected_review_list

def write_into_file(file_path: str, data: list):
	with open(file_path, mode="w") as file:
		file.write("\n".join(data))



X_train_treated = pretreatment(dataset_train)
random.shuffle(X_train_treated)
write_into_file("X_train.txt", X_train_treated)

(2 points) Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

In [ ]:
model = fasttext.train_supervised(input="X_train.txt")

In [ ]:
dataset_test = load_dataset("imdb", split="test")
X_test_treated = pretreatment(dataset_test)
random.shuffle(X_test_treated)
X_test_treated[:5]

In [ ]:
write_into_file("X_test.txt",X_test_treated)
model.test("X_test.txt")

In [ ]:
model.test("X_train.txt")

(3 points) Use the hyperparameters search functionality of FastText and repeat step 2.

    To do so, you'll need to split your training set into a training and a validation set (don't forget to shuffle your dataset).
    Let the model search for 5 minutes (it's the default search time).


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_vallidation = train_test_split(X_train_treated, shuffle=True)
write_into_file("X_train.txt",X_train)
write_into_file("X_val.txt", X_vallidation)

In [ ]:
model2 = fasttext.train_supervised(input="X_train.txt",autotuneValidationFile="X_val.txt")
model2.test("X_test.txt")

 (1 points) Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ?

    Check the Tips on using FastText section for a bit of help).
    Only refer to the attributes you think are interesting.
    See the Tips on using FastText (just below) for help.


In [ ]:
print("	   Model1 |  Model2 ")
def my_print(name: str, val1: str, val2 :str):
	print(f"{name}:  {val1}    | {val2}")
my_print("Epoch",model.epoch,model2.epoch)
my_print("Dim",model.dim,model2.dim) #Pas sur que ce soit intérréssant
my_print("Bucket",model.bucket,model2.bucket)
my_print("Loss function", model.loss,model2.loss)

(1 point) Using the tuned model, take at least 2 wrongly classified examples from the test set, and try explaining why the model failed.

In [ ]:
Y_test = model2.predict(X_test_treated)

In [ ]:
for i,j in enumerate(X_test_treated):
	print(i,j[9:16])

In [ ]:
print(X_test_treated[11],Y_test[0][11])
print(X_test_treated[15],Y_test[0][15])
print(X_test_treated[38],Y_test[0][38])